# Gating Model - Expert Ensemble
Reference: how-can-we-prevent-road-rage (merging outputs)

This notebook implements the gating model that combines outputs from all expert models.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

print("Gating Model - Expert Ensemble - Ready for implementation")

## Expert Model Integration

Combine outputs from:
1. Behavior Expert
2. Geographic Expert
3. Contextual Expert

In [ ]:
class ExpertEnsemble:
    """
    Ensemble model combining expert outputs
    """
    
    def __init__(self):
        self.expert_weights = {
            'behavior': 0.4,
            'geographic': 0.3,
            'contextual': 0.3
        }
        
        self.risk_thresholds = {
            'low': 30,
            'moderate': 60,
            'high': 80
        }
    
    def combine_expert_scores(self, behavior_score, geo_risk, context_risk):
        """
        Weighted combination of expert scores
        """
        # Convert scores to same scale (risk scores)
        behavior_risk = 100 - behavior_score  # Invert behavior score
        
        # Weighted average
        combined_risk = (
            behavior_risk * self.expert_weights['behavior'] +
            geo_risk * self.expert_weights['geographic'] +
            context_risk * self.expert_weights['contextual']
        )
        
        return {
            'combined_risk': combined_risk,
            'risk_category': self._categorize_risk(combined_risk),
            'expert_contributions': {
                'behavior': behavior_risk,
                'geographic': geo_risk,
                'contextual': context_risk
            }
        }
    
    def _categorize_risk(self, risk_score):
        """Categorize risk level"""
        if risk_score < self.risk_thresholds['low']:
            return "Low Risk"
        elif risk_score < self.risk_thresholds['moderate']:
            return "Moderate Risk"
        elif risk_score < self.risk_thresholds['high']:
            return "High Risk"
        else:
            return "Very High Risk"

# Test the ensemble
ensemble = ExpertEnsemble()
test_result = ensemble.combine_expert_scores(
    behavior_score=85,
    geo_risk=45,
    context_risk=55
)
print(f"Ensemble test result: {test_result}")

## Premium Calculation

Convert risk scores to insurance premium adjustments

In [ ]:
def calculate_premium_adjustment(risk_score, base_premium=1000):
    """
    Calculate insurance premium based on combined risk score
    """
    # Premium adjustment curve
    if risk_score < 30:
        adjustment_factor = 0.8  # 20% discount
        tier = "Preferred"
    elif risk_score < 50:
        adjustment_factor = 0.9  # 10% discount
        tier = "Standard Plus"
    elif risk_score < 70:
        adjustment_factor = 1.0  # Standard rate
        tier = "Standard"
    elif risk_score < 85:
        adjustment_factor = 1.2  # 20% surcharge
        tier = "Substandard"
    else:
        adjustment_factor = 1.5  # 50% surcharge
        tier = "High Risk"
    
    adjusted_premium = base_premium * adjustment_factor
    savings = base_premium - adjusted_premium
    
    return {
        'base_premium': base_premium,
        'adjusted_premium': adjusted_premium,
        'adjustment_factor': adjustment_factor,
        'savings': savings,
        'tier': tier
    }

# Test premium calculation
premium_result = calculate_premium_adjustment(test_result['combined_risk'])
print(f"Premium calculation: {premium_result}")

## Model Performance Analysis

Analyze expert model contributions and performance

In [ ]:
def analyze_expert_contributions(expert_scores_df):
    """
    Analyze how each expert model contributes to final scores
    """
    # Calculate correlations between expert scores
    correlations = expert_scores_df[['behavior', 'geographic', 'contextual']].corr()
    
    # Feature importance (simplified)
    importance = {
        'behavior': expert_scores_df['behavior'].std(),
        'geographic': expert_scores_df['geographic'].std(),
        'contextual': expert_scores_df['contextual'].std()
    }
    
    return {
        'correlations': correlations,
        'variability': importance
    }

# Create sample data for analysis
sample_data = pd.DataFrame({
    'behavior': np.random.normal(75, 15, 100),
    'geographic': np.random.normal(50, 20, 100),
    'contextual': np.random.normal(45, 18, 100)
})

analysis_result = analyze_expert_contributions(sample_data)
print("Expert contribution analysis completed")
print(f"Correlations:\n{analysis_result['correlations']}")

## Visualization

Visualize expert model outputs and ensemble results

In [ ]:
def plot_expert_distributions(expert_scores_df):
    """
    Plot distributions of expert model scores
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Individual expert distributions
    expert_scores_df['behavior'].hist(ax=axes[0,0], bins=20, alpha=0.7)
    axes[0,0].set_title('Behavior Scores')
    
    expert_scores_df['geographic'].hist(ax=axes[0,1], bins=20, alpha=0.7)
    axes[0,1].set_title('Geographic Risk Scores')
    
    expert_scores_df['contextual'].hist(ax=axes[1,0], bins=20, alpha=0.7)
    axes[1,0].set_title('Contextual Risk Scores')
    
    # Correlation heatmap
    sns.heatmap(expert_scores_df[['behavior', 'geographic', 'contextual']].corr(), 
                annot=True, ax=axes[1,1], cmap='coolwarm', center=0)
    axes[1,1].set_title('Expert Score Correlations')
    
    plt.tight_layout()
    plt.show()
    
    print("Expert distribution plots created")

# Create visualization
plot_expert_distributions(sample_data)

## Model Optimization

Optimize expert weights based on performance data

In [ ]:
def optimize_expert_weights(expert_scores, actual_claims):
    """
    Optimize expert weights based on claim prediction accuracy
    """
    # This would be implemented with actual claims data
    # Using optimization algorithms to find best weights
    
    from scipy.optimize import minimize
    
    def objective(weights):
        # Calculate combined scores with given weights
        combined_scores = (
            expert_scores['behavior'] * weights[0] +
            expert_scores['geographic'] * weights[1] +
            expert_scores['contextual'] * weights[2]
        )
        
        # Return negative correlation (to maximize)
        correlation = np.corrcoef(combined_scores, actual_claims)[0, 1]
        return -correlation if not np.isnan(correlation) else 1
    
    # Constraints: weights sum to 1
    constraints = {'type': 'eq', 'fun': lambda x: sum(x) - 1}
    bounds = [(0, 1), (0, 1), (0, 1)]
    
    # Initial weights
    initial_weights = [0.4, 0.3, 0.3]
    
    # Note: This is a placeholder - would use real claims data
    print("Weight optimization framework ready (requires real claims data)")
    
    return initial_weights

# Test optimization framework
optimized_weights = optimize_expert_weights(sample_data, np.random.random(100))
print(f"Optimized weights: {optimized_weights}")